In [ ]:

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
def dat_list():# create list of data type based on the descrption 
    categ=['MSSubClass','MSZoning','Street','Alley',"LotShape","LandContour","Utilities","LotConfig",\
      "LandSlope","Neighborhood","Condition1","Condition2","BldgType","HouseStyle","OverallQual",\
      "OverallCond",'RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual',\
      'ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',\
      "Heating","HeatingQC","Electrical",'KitchenQual','Functional','FireplaceQu','GarageType',\
      'GarageFinish','GarageCars','GarageQual','GarageCond','PavedDrive','PoolQC','Fence','MiscFeature',\
       'SaleType','SaleCondition','CentralAir']
    # note categorical include binary and categ_usefulNA 


       
    binary=['HeatingQC','CentralAir'] # binary categorical
    categ_usefulNA=["Alley",'BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType',\
                     'GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature']
    #categorical that have NA with useful meaning(instead of missing)
   
    numerical=['GrLivArea','LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF',\
          'GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea',\
          'MiscVal','1stFlrSF','2ndFlrSF','LowQualFinSF']
   
    #note that numerical do not have year or int_data
    Year=['YearBuilt','YearRemodAdd','GarageYrBlt','YrSold']

    int_data=['BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr',\
          'TotRmsAbvGrd',"Fireplaces",'MoSold']

  
    #list of categorical variable, but NA is not missing data.
    return categ, binary, numerical, Year, int_data, categ_usefulNA

In [ ]:
def correct_data_type(data): # correct the datatype according to the data description
    data[data.columns.intersection(dat_list()[0])]=data[data.columns.intersection(dat_list()[0])].astype('category')
    # using the list from dat_list to find columns
    data[data.columns.intersection(dat_list()[2])]=data[data.columns.intersection(dat_list()[2])].astype('float')
    data[data.columns.intersection(dat_list()[3])]=data[data.columns.intersection(dat_list()[3])].astype('int',errors='ignore')#
    data[data.columns.intersection(dat_list()[4])]=data[data.columns.intersection(dat_list()[4])].astype('int',errors='ignore')
    return data



def replace_na(df):
    df[dat_list()[5]]=df[df.columns.intersection(dat_list()[5])].replace(float('nan'),'Donthave')
    # these are the variables that has useful Na
    for column in dat_list()[0]: # replace categorical with mode()
        df[column].fillna(df[column].mode()[0], inplace=True)
    for column in dat_list()[2]: # replace float with median
        df[column].fillna(df[column].median(), inplace=True)
    for column in dat_list()[3]: # replace year with median 
        df[column].fillna(df[column].median(), inplace=True)
    for column in dat_list()[4]: # replace integer value with median 
        df[column].fillna(df[column].median(), inplace=True)
    train=df.dropna()
    test=df[df["SalePrice"].isna()!=False]
    return train, test, df

#complete_data=correct_data_type(complete_data)
#complete_data=replace_na(complete_data)[2]
#complete_data

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt 
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import normalize
from sklearn.preprocessing import OrdinalEncoder
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as stats
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from numpy import array 



def constant_exclude(dataset,n):
    Data=dataset                 #Import the complete set
    numeric=pd.concat([Data[Data.columns&dat_list()[2]],Data[Data.columns&dat_list()[3]],Data[Data.columns&dat_list()[4]]],axis=1) # get only numeric variables
    numeric_normal=normalize(numeric) #Normalize the values of numeric variables
    sel = VarianceThreshold(threshold=n) #Determine the threshold
    sel.fit(numeric_normal) #Variance lower than the threshold get removed.
    value=sel.get_support() 
    reduced_numeric=numeric.loc[:,value] #Get remaining columns
    complete_set=pd.concat([Data['SalePrice'],Data['Id'],reduced_numeric,Data[Data.columns&dat_list()[0]]],axis=1) #Add back the categorical columns, "Id" and "Sale Price".
    return complete_set, sum(value)   #Return complete set, number of remaining columns.

def k_best(data,n):
    Data=data  #Import the complete set
    train=data[data['SalePrice'].isna()==False] #get the traininng data
    numeric=dat_list()[2]+dat_list()[3]+dat_list()[4] #Get the numeric variables.
    numeric=train[train.columns.intersection(numeric)].copy() 
    New = SelectKBest(f_regression, k=n) #Select variables with highest kth F-statistic score
    New.fit_transform(numeric, train['SalePrice']) 
    Filter = New.get_support()
    Reduced_numeric=numeric.loc[:,Filter] #Get remaining columns
    output=Data[Reduced_numeric.columns]
    output['SalePrice']=Data['SalePrice'] #Add back Saleprice
    output['Id']=Data['Id'] #Add back "Id"
    categorical= dat_list()[0]
    output[categorical]=Data[categorical] #Add back categorical variables.
    return  output

def cate_to_dummy(data):
    Data=data
    dummy_set=pd.get_dummies(Data[Data.columns.intersection(dat_list()[0])]) #Transform categorical data to dummy variables
    total=pd.concat([Data['SalePrice'],Data['Id'],dummy_set,Data[Data.columns&dat_list()[2]],Data[Data.columns&dat_list()[3]],Data[Data.columns&dat_list()[4]]],axis=1) #Add back the numeric variables and "Id" and "Sale Price"
    return total



#constant_exclude(complete_data,0.002)[0]
#complete_data=k_best(complete_data,12)
#complete_data=cate_to_dummy(complete_data)


In [ ]:

from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

def forward_select(data):
    Data=data
    lr = LinearRegression()
    # After test the numbers of k_features = 40 satisfy our purpose
    sfs = SFS(lr, k_features = 40, forward = True, floating = False, verbose = 2 #get details
              ,scoring = 'r2', cv = 10 ) #neg_mean_squared_error to be consider
    train = Data[0:1460]
    test = Data[1460:2920]
    y_train = train['SalePrice'].to_numpy()
    y_train = y_train.reshape(-1,1)
    X_train = train.drop('SalePrice',axis = 1)
    X_test = test.drop('SalePrice',axis = 1)
    sfs.fit(X_train,y_train)
    x_variable = list(sfs.k_feature_names_)
    X_train = X_train[x_variable]
    lr.fit(X_train,y_train)
    X_test = X_test[x_variable].to_numpy()
    y_pred = lr.predict(X_test)
    id = np.array([range(1461,2920)])
    Id = pd.DataFrame(id.reshape(-1,1), columns = ['Id'])
    saleprice = pd.DataFrame(y_pred, columns = ['SalePrice'])
    total = pd.concat([Id,saleprice],axis = 1)

    return Data[x_variable], total

def stepwise_select(data):
    Data=data
    lr = LinearRegression()
    # set up the train
    train = Data[0:1460]
    test = Data[1460:2920]
    y_train = train['SalePrice'].to_numpy()
    y_train = y_train.reshape(-1,1)
    X_train = train.drop('SalePrice',axis = 1)
    X_test = test.drop('SalePrice',axis = 1)

    # After test the numbers of k_features = 40 satisfy our purpose
    sffs = SFS(LinearRegression(), k_features = (1,40), forward = True, floating = True,verbose = 2,
              scoring = 'r2', cv = 10 )
    sffs.fit(X_train,y_train)
    x_variable = list(sffs.k_feature_names_)
    # updated the selected features to do linear regression prediction
    X_train = X_train[x_variable]
    
    lr.fit(X_train,y_train)
    X_test = X_test[x_variable]
    y_pred = lr.predict(X_test)
    id = np.array([range(1461,2920)])
    Id = pd.DataFrame(id.reshape(-1,1), columns = ['Id'])
    saleprice = pd.DataFrame(y_pred, columns = ['SalePrice'])
    total = pd.concat([Id,saleprice],axis = 1)
    
    return Data[x_variable], total

def backward_select(data):
    lr = LinearRegression()
    Data=data
    # After test the numbers of k_features = 40 satisfy our purpose
    sbs = SFS(lr, k_features = 40, forward = False, floating = False,verbose = 2,
              scoring = 'r2', cv = 10 )
    # set up the train
    train = Data[0:1460]
    test = Data[1460:2920]
    y_train = train['SalePrice'].to_numpy()
    y_train = y_train.reshape(-1,1)
    X_train = train.drop('SalePrice',axis = 1)
    X_test = test.drop('SalePrice',axis = 1)
    sbs.fit(X_train,y_train)
    x_variable = list(sbs.k_feature_names_)
    # Update X_train
    X_train = X_train[x_variable]
    X_test = X_test[x_variable]
    lr.fit(X_train,y_train)
    y_pred = lr.predict(X_test)
    id = np.array([range(1461,2920)])
    Id = pd.DataFrame(id.reshape(-1,1), columns = ['Id'])
    saleprice = pd.DataFrame(y_pred, columns = ['SalePrice'])
    total = pd.concat([Id,saleprice],axis = 1)
    return Data[x_variable], total

#backward_select(complete_data)[1]

In [ ]:
import sklearn.linear_model as lm
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

def generate_xy(df):# you should input the complete dataset here.
    
    training=df[df['SalePrice'].isna()==False] # for training, the Saleprice should not be missing
    training
    testing=df[df['SalePrice'].isna()==True] # for testing, the Saleprice should be NA's, use this to track testing

    y_train=training['SalePrice']
    x_train= training.drop(['Id','SalePrice'],axis=1)
    x_test= testing.drop(['SalePrice'],axis=1)
    # for x_test, we want to keep the id for tracking the output id in the final stage 
    return x_train,y_train, x_test
    


def lass_CV(df,xtrain,ytrain,xtest):
    Id=xtest['Id']
    xtest= xtest.drop(['Id'],axis=1)
    
    numerical_dat= dat_list()[2]+dat_list()[3]+dat_list()[4] # find all numerical data
    numerical_dat=xtrain.columns.intersection(numerical_dat) 
    # because we might delete some columns in filter methods, so we use intersection to find the columns still left.
    scaler= StandardScaler() # scale the data first before fitting
    xtrain[numerical_dat]= scaler.fit_transform(xtrain[numerical_dat])
    xtest[numerical_dat] = scaler.fit_transform(xtest[numerical_dat])
    model = lm.LassoCV(cv=10, random_state=0, max_iter=10000) # use cross validation score to find penalty term
    model.fit(xtrain, ytrain)
    model.alpha_
    lasso_best = lm.Lasso(alpha=model.alpha_) # fit best model with best alpha
    lasso_best.fit(xtrain, ytrain)
    result=lasso_best.predict(xtest)
    output=pd.DataFrame()
    output['Id']= Id
    output['Saleprice']=result # Kaggle requires the result to have 2 columns: Id and Saleprice
    return output  

def Rid_CV(df,xtrain,ytrain,xtest):
    # ridge is very similar to lasso
    Id=xtest['Id']
    xtest= xtest.drop(['Id'],axis=1)
    
    numerical_dat= dat_list()[2]+dat_list()[3]+dat_list()[4]
    numerical_dat=xtrain.columns.intersection(numerical_dat)
    scaler= StandardScaler()
    xtrain[numerical_dat]= scaler.fit_transform(xtrain[numerical_dat])
    xtest[numerical_dat] = scaler.fit_transform(xtest[numerical_dat])
    model = lm.RidgeCV(cv=10)
    model.fit(xtrain, ytrain)
    ridge_best = lm.Ridge(alpha=model.alpha_)
    ridge_best.fit(xtrain, ytrain)
    result=ridge_best.predict(xtest)
    output=pd.DataFrame()
    output['Id']= Id
    output['Saleprice']=result
    return output

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor



def random_forest_default(df,xtrain,ytrain,xtest):
        Id=xtest['Id']
        xtest= xtest.drop(['Id'],axis=1)
        rf = RandomForestRegressor() # call the default model 
        rf.fit(xtrain,ytrain)
        result= rf.predict(xtest) # use default model to train and predict 
        output=pd.DataFrame()
        output['Id']= Id
        output['Saleprice']=result
        return output
             
def rf_cv_tun():# this function does a automatic tunning for random forest
    rf=RandomForestRegressor()
    n_estimators = [1000,2000,3000]
    max_features = ['auto', 'sqrt']
    max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
    min_samples_split = [x/10 for x in np.linspace(2, 10, num = 5)]
    min_samples_leaf = [1, 2, 4]
    bootstrap = [True, False]

    random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap} # input values into the parameter space
    rf = RandomForestRegressor()
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5,random_state=30, n_jobs = -1)
    # use random search and cross validation to find best combinations in parameter space. 
    return rf_random


def random_forest_best(df,xtrain,ytrain,xtest,threshold,model):
    #threshold is the feature score for which you want the features to be selected.
    # use model from rf_cv_tun(), a model that tuns parameters with CV
    
    Id=xtest['Id']
    xtest= xtest.drop(['Id'],axis=1)
    
    model.fit(xtrain,ytrain)
    feature_scores = pd.Series(model.best_estimator_.feature_importances_, index=xtrain.columns).sort_values(ascending=False)
    #find the feature score of the model
    important_feature=feature_scores.index.where(feature_scores>threshold).dropna()
    # filer variables that has feature importance above threshold
    # select those variables in x train and xtest
    xtrain=xtrain[important_feature]
    xtest=xtest[important_feature]
    model.fit(xtrain,ytrain)
    result=model.predict(xtest)
    output=pd.DataFrame()
    output['Id']= Id
    output['Saleprice']=result
    return output

import xgboost as xgb

from sklearn.model_selection import RandomizedSearchCV

from xgboost.sklearn import XGBRegressor



def rand_cv_tunning(): # a function that tuns the XGB model
    parameters = {'objective':['reg:squarederror'],
                  'booster':['gbtree'], # can be gbtree or linear, but generally trees have better performance
                  'learning_rate': [x for x in np.linspace(start = 0.05, stop = 0.3, num = 6)], 
                  # a number between 0 to 1, determines the learning rate
                  'max_depth': [5,10,15,20],
                  'min_child_weight': [10,15,20,25],
                 'colsample_bytree': [i/10 for i in range(1,10)],
                  'n_estimators': [1000,2000]}

    xgb_model = XGBRegressor(random_state=30)
    xgb = RandomizedSearchCV(xgb_model, parameters, cv=5,n_iter=100,scoring='neg_mean_squared_error',n_jobs=-1,random_state=30)
    return xgb

#xgb_model=rand_cv_tunning()



def xgb_default(complete_data,xtrain,ytrain,xtest):
    Id=xtest['Id']
    xtest= xtest.drop(['Id'],axis=1)
    xgb_model = XGBRegressor(random_state=30)
    xgb_model.fit(xtrain,ytrain)
    result= xgb_model.predict(xtest)
    
    output=pd.DataFrame()
    output['Id']= Id
    output['Saleprice']=result
    
    return output



def XGB_with_tun(df,xtrain, ytrain, xtest,model):# use model from rand_cv_tunning()
       Id=xtest['Id']
        xtest= xtest.drop(['Id'],axis=1)
        
        model.fit(xtrain,ytrain)
        result=model.predict(xtest)
        output=pd.DataFrame()
        output['Id']= Id
        output['Saleprice']=result
        return output

In [ ]:
# run code
wd=os.getcwd()

path=wd+'/house_price data/train.csv'
Path=wd+'/house_price data/test.csv'

train_data=pd.read_csv(path)
test_data=pd.read_csv(Path)
train_data= pd.read_csv('../142A final project/train.csv')
train_data=pd.DataFrame(train_data)

test_data=pd.read_csv('../142A final project/test.csv')
test_data=pd.DataFrame(test_data)

#complete_data= pd.concat([train_data,test_data])
#complete_data=correct_data_type(complete_data)
#complete_data=replace_na(complete_data)[2]
#complete_data
#wd

In [ ]:
#trail 1
# It goes through all wrapper methods and embedded methods with data filtered from filter methods 
data_1=complete_data
data_1=k_best(data_1,12)
data_1=constant_exclude(data_1,0.002)[0]
data_1=cate_to_dummy(data_1)


output1=forward_select(data_1)[1]
output1.to_csv('/testing_1.csv', index = False) # output the result to CSV file

output2=backward_select(data_1)[1]
output2.to_csv('/testing_2.csv', index = False)

output3=stepwise_select(data_1)[1]
output3.to_csv('/testing_3.csv', index = False)


x_train, y_train, x_test= generate_xy(data_1)
output4=lass_CV(data_1,x_train,y_train,x_test)
output4.to_csv('/testing_4.csv', index = False)

output5=Rid_CV(data_1,x_train,y_train,x_test)
output5.to_csv('/testing_5.csv', index = False)


output6=random_forest_default(data_1,x_train,y_train,x_test)
output6.to_csv('/testing_6.csv', index = False)


output7=xgb_default(data_1,x_train,y_train,x_test)
output7.to_csv('/testing_7.csv', index = False)

rf_model= rf_cv_tun()
output8=random_forest_best(data_1,x_train,y_train,x_test,0.0001,rf_model)
output8.to_csv('/testing_8.csv', index = False)

xgb_model=rand_cv_tunning()
output9=XGB_with_tun(data_1,x_train,y_train,x_test,xgb_model)
output9.to_csv('/testing_9.csv', index = False)

In [ ]:
data_2=complete_data # this trail goes through the wrapper and embbeded methods without filter method

data_2=cate_to_dummy(data_2)


output10=forward_select(data_2)[1]
output10.to_csv('/Users/lizehao727/Downloads/testing_10.csv', index = False)

output11=backward_select(data_2)[1]
output11.to_csv('/Users/lizehao727/Downloads/testing_11.csv', index = False)

output12=stepwise_select(data_2)[1]
output12.to_csv('/Users/lizehao727/Downloads/testing_12.csv', index = False)


x_train, y_train, x_test= generate_xy(data_2)

output13=lass_CV(data_2,x_train,y_train,x_test)
output13.to_csv('/Users/lizehao727/Downloads/testing_13.csv', index = False)

output14=Rid_CV(data_2,x_train,y_train,x_test)
output14.to_csv('/Users/lizehao727/Downloads/testing_14.csv', index = False)


output15=random_forest_default(data_2,x_train,y_train,x_test)
output15.to_csv('/testing_15.csv', index = False)


output16=xgb_default(data_2,x_train,y_train,x_test)
output16.to_csv('/testing_16.csv', index = False)

rf_model= rf_cv_tun()
output17=random_forest_best(data_2,x_train,y_train,x_test,0.0001,rf_model)
output17.to_csv('/testing_17.csv', index = False)

xgb_model=rand_cv_tunning()
output18=XGB_with_tun(data_2,x_train,y_train,x_test,xgb_model)
output18.to_csv('/testing_18.csv', index = False)

In [ ]:
#The following are plotting codes. Adjustment for variable name is required.

#Variance bar plot:
#data=remove_na(complete_data)[2]
#Data=replace_na(data)[2]
#numeric=pd.concat([Data[Data.columns&dat_list()[2]],Data[Data.columns&dat_list()[3]],Data[Data.columns&dat_list()[4]]],axis=1) #numeric variables
#numeric_normal=normalize(numeric)
#numeric_normal=pd.DataFrame(numeric_normal, columns=numeric.columns)
#var=np.array(numeric_normal.var()).reshape(1,32)
#Var=pd.DataFrame(var,columns=numeric.columns,index=['Variance_for_each_feature'])
#Var=Var.sort_values(by='Variance_for_each_feature',axis=1,ascending=False)
#Var.plot.bar(rot=0)
#numeric_normal.var().sort_values(ascending=False)

#F-statistic score bar plot:
#Data=replace_na(trainset)[2]
#numeric=pd.concat([Data[Data.columns&dat_list()[2]],Data[Data.columns&dat_list()[3]],Data[Data.columns&dat_list()[4]]],axis=1)
#vvalue=f_regression(numeric,trainset['SalePrice'])
#f_stat=pd.DataFrame(vvalue, columns=numeric.columns,index=['F_statistic','p_value'])
#f_stat=f_stat.drop(labels='p_value',axis=0)
#F_stat=f_stat.sort_values(by='F_statistic',axis=1,ascending=False)
#F_stat['KitchenAbvGr']
#F_stat.plot.bar(rot=0)
#sum(F_stat.iloc[0]>27)
